# Transfer the pick data

As we reorder the gouge event catalog when merging the main set of events and the tiny events, we transfer the pick files of the P wave first arrivals.

2024.10.30 Kurama Okubo

In [1]:
import os
import shutil

import glob
import pandas as pd

In [2]:
# Read new merged catalog of the gouge events

df_event = pd.read_csv("../../Experiments/DetectEvent/data/p06_visual_pick_gougeevents_merged.csv", header=4, 
            names=["expr_id", "event_id", "gouge_loc", "picktime", "eventtype", "rup_type", "gougeevent_id",
                   "doublecheck", "old_gougeevent_id"])

df_event.head()

,expr_id,event_id,gouge_loc,picktime,eventtype,rup_type,gougeevent_id,doublecheck,old_gougeevent_id
0,fb03-087,2,2.75,41.02500,LFE,2,0,1,0
1,fb03-087,3,1.25,42.46400,LFE,1,1,1,1
2,fb03-087,3,1.25,42.46450,Ordinary,1,2,1,2
3,fb03-087,3,3.25,42.46790,Ordinary,1,3,1,212
4,fb03-087,3,1.75,42.47475,Ordinary,1,4,1,3


In [3]:
# filter the event type
df_event_filtered = df_event[(df_event["eventtype"]=="Ordinary") & (df_event["gouge_loc"]==1.75)]

In [4]:
df_event_filtered.head()

,expr_id,event_id,gouge_loc,picktime,eventtype,rup_type,gougeevent_id,doublecheck,old_gougeevent_id
4,fb03-087,3,1.75,42.47475,Ordinary,1,4,1,3
9,fb03-087,5,1.75,47.66270,Ordinary,1,9,1,201
18,fb03-087,8,1.75,56.72700,Ordinary,1,18,1,16
19,fb03-087,9,1.75,59.10315,Ordinary,1,19,1,202
20,fb03-087,9,1.75,59.10350,Ordinary,1,20,1,17


In [5]:
# dump gougeevent id
df_event_filtered.gougeevent_id.values


array([  4,   9,  18,  19,  20,  21,  24,  27,  30,  31,  37,  38,  40,
        43,  44,  45,  49,  50,  52,  55,  59,  61,  62,  69,  72,  75,
        76,  77,  81,  85,  88,  89,  95,  99, 100, 102, 109, 110, 111,
       118, 120, 126, 128, 129, 131])

In [6]:
arrivalpick_root = "../data/AElocation/arrivalpick/"
if not os.path.exists(arrivalpick_root):
    os.makedirs(arrivalpick_root)

# 1. transfer the arrival pick
We transfer the pick file made by `01_AE_locateevent_GUI.py`.

In [7]:
pickdir_main = "../../../4mBIAX_Paper/Processing/SourceInversion/data/AElocation/arrivalpick"
pickdir_tiny = "../../../4mBIAX_Paper/Processing/AnalyzeTinyEvents/waveforms_inversion/data/AElocation/arrivalpick"

new_events = [] # We added new events for the merged catalog, which we need to pick.

for i, row in df_event_filtered.iterrows():
    # row = df_event_filtered.iloc[10]
    old_id = row.old_gougeevent_id
    new_id = row.gougeevent_id
    
    if old_id < 200:
        # main event set
        srcdir = pickdir_main+f"/{old_id:04d}"
    else:
        # tiny event set
        srcdir = pickdir_tiny+f"/{old_id-201:04d}"
    
    # if the file is not found, those events have been newly added to the merged catalog.
    
    if not os.path.exists(srcdir):
        new_events.append(new_id)
        
    # copy the folder and rename
    else:
        new_pickdir = f"../data/AElocation/arrivalpick/{new_id:04d}"
        if not os.path.exists(new_pickdir):
            os.makedirs(new_pickdir)
        for fi in glob.glob(srcdir+"/*.pickle"):
            shutil.copyfile(fi, new_pickdir+f"/{os.path.basename(fi).replace(f'{old_id:04d}', f'{new_id:04d}')}")

# glob.glob(pickdir_main+"/*")
#

In [8]:
print("new events to be picked are follows:", new_events)

new events to be picked are follows: [30, 45, 59, 75, 76, 77]


# Transfer the repick file
We also transfer the repick files.

In [9]:
repickdir_main = "../../../4mBIAX_Paper/Processing/SourceInversion/data/AElocation/relocation"
repickdir_tiny = "../../../4mBIAX_Paper/Processing/AnalyzeTinyEvents/waveforms_inversion/data/AElocation/relocation"

new_events = [] # We added new events for the merged catalog, which we need to pick.

for i, row in df_event_filtered.iterrows():
    # row = df_event_filtered.iloc[10]
    old_id = row.old_gougeevent_id
    new_id = row.gougeevent_id
    
    if old_id < 200:
        # main event set
        srcdir = repickdir_main+f"/{old_id:04d}"
    else:
        # tiny event set
        srcdir = repickdir_tiny+f"/{old_id:04d}"
    
    # if the file is not found, those events have been newly added to the merged catalog.
    
    if not os.path.exists(srcdir):
        new_events.append(new_id)
        
    # copy the folder and rename
    else:
        new_pickdir = f"../data/AElocation/relocation/{new_id:04d}"
        if not os.path.exists(new_pickdir):
            os.makedirs(new_pickdir)
        for fi in glob.glob(srcdir+"/*.pickle"):
            shutil.copyfile(fi, new_pickdir+f"/{os.path.basename(fi).replace(f'{old_id:04d}', f'{new_id:04d}')}")

# glob.glob(pickdir_main+"/*")
#